In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import time
import os

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

In [ ]:
def print_separator(title=""):
    """구분선 출력"""
    print("\n" + "="*80)
    if title:
        print(f" {title}")
        print("="*80)
    print()

def print_parameters(params, title):
    """파라미터 출력"""
    print(f" {title} 파라미터:")
    for key, value in params.items():
        print(f" {key}: {value}")
    print()

# 모델 파라미터 설정 (Before / After)
before_params = {
    "temperature": 0.7,         # 높은 창의성 (다양한 답변)
    "max_tokens": 300,          # 짧은 응답 길이
    "frequency_penalty": 0.5,   # 높은 반복 억제
    "presence_penalty": 0.5,    # 높은 새로운 단어 장려
}

after_params = {
    "temperature": 0.3,         # 낮은 창의성 (정확한 답변)
    "max_tokens": 800,          # 긴 응답 길이
    "top_p": 0.85,              # 확률 기반 샘플링 추가
    "frequency_penalty": 0.2,   # 낮은 반복 억제 (자연스러운 답변)
    "presence_penalty": 0.3,    # 낮은 새로운 단어 장려
}

# 파라미터 정보 출력
print_separator("모델 파라미터 비교")
print_parameters(before_params, "BEFORE")
print_parameters(after_params, "AFTER")

In [ ]:
# 두 개의 모델 생성 (Before / After)
print(" 모델 초기화 중...")

try:
    # Before 모델 (기본적인 설정)
    before_model = ChatOpenAI(
        base_url="https://api.groq.com/openai/v1",
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        **before_params
    )
    
    # After 모델 (최적화된 설정) - 변수명 오타 수정
    after_model = ChatOpenAI(
        base_url="https://api.groq.com/openai/v1",
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        **after_params
    )
    
    print(" 모델 초기화 완료")
    
except Exception as e:
    print(f" 모델 초기화 실패: {e}")
    print("API 키를 확인하세요.")
    exit(1)

# 프롬프트 구성
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

# 체인 생성
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

# 동일한 질문으로 비교 (파라미터 차이 효과 확인)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

print(" 동일한 질문으로 파라미터 차이 비교:")
print(f"질문: {test_question}")

In [ ]:
def execute_with_timing(chain, question, model_name):
    """시간 측정과 함께 모델 실행"""
    print(f"\n {model_name} 모델 실행 중...")
    start_time = time.time()
    
    try:
        response = chain.invoke({"user_input": question})
        end_time = time.time()
        execution_time = end_time - start_time
        
        print(f" 실행 시간: {execution_time:.2f}초")
        print(f" 응답 길이: {len(response.content)}자")
        
        return response, execution_time
        
    except Exception as e:
        print(f" 실행 실패: {e}")
        return None, 0

# Before 모델 실행
print_separator("BEFORE 모델 결과")
before_response, before_time = execute_with_timing(before_chain, test_question, "BEFORE")

if before_response:
    print("\n 응답 내용:")
    print("-" * 40)
    print(before_response.content)

# After 모델 실행  
print_separator("AFTER 모델 결과")
after_response, after_time = execute_with_timing(after_chain, test_question, "AFTER")

if after_response:
    print("\n 응답 내용:")
    print("-" * 40)
    print(after_response.content)

# 비교 분석
print_separator("성능 비교 분석")

if before_response and after_response:
    print(" 성능 지표 비교:")
    print(f"   실행 시간: BEFORE {before_time:.2f}초 vs AFTER {after_time:.2f}초")
    print(f"   응답 길이: BEFORE {len(before_response.content)}자 vs AFTER {len(after_response.content)}자")
    
    # 응답 길이 차이 분석
    length_diff = len(after_response.content) - len(before_response.content)
    if length_diff > 0:
        print(f"    AFTER가 {length_diff}자 더 길어짐 (더 상세한 정보)")
    elif length_diff < 0:
        print(f"    AFTER가 {abs(length_diff)}자 더 짧아짐 (더 간결한 정보)")
    else:
        print(f"    두 응답의 길이가 동일함")
    
    # 시간 차이 분석
    time_diff = after_time - before_time
    if time_diff > 0:
        print(f"    AFTER가 {time_diff:.2f}초 더 소요됨")
    elif time_diff < 0:
        print(f"    AFTER가 {abs(time_diff):.2f}초 더 빨름")
    else:
        print(f"    실행 시간이 거의 동일함")

# 추가 분석: 다양한 질문으로 테스트
print_separator("추가 테스트: 다양한 질문")

additional_questions = [
    "부산 1일 여행 코스를 추천해 주세요.",
    "제주도에서 가족이 즐길 수 있는 액티비티는?",
    "서울에서 한옥마을 체험할 수 있는 곳은?"
]

for i, question in enumerate(additional_questions, 1):
    print(f"\n테스트 {i}: {question}")
    
    # Before 모델
    before_resp, before_t = execute_with_timing(before_chain, question, f"BEFORE-{i}")
    
    # After 모델  
    after_resp, after_t = execute_with_timing(after_chain, question, f"AFTER-{i}")
    
    if before_resp and after_resp:
        print(f"    길이 차이: {len(after_resp.content) - len(before_resp.content)}자")
        print(f"    시간 차이: {after_t - before_t:.2f}초")